In [1]:
# ====================================================
# import
# ====================================================
import operator
from typing import Annotated, Any, Optional

from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph
from pydantic import BaseModel, Field


## データ構造の定義

In [2]:
# ====================================================
# データモデル
# ====================================================

# ペルソナを表すデータモデル
class Persona(BaseModel):
    name: str = Field(description="ペルソナの名前")
    background: str = Field(description="ペルソナの持つ背景")


# ペルソナのリストを表すデータモデル
class Personas(BaseModel):
    personas: list[Persona] = Field(default_factory=list, description="ペルソナのリスト")


# インタビュー内容を表すデータモデル
class Interview(BaseModel):
    persona: Persona = Field(description="インタビュー対象のペルソナ")
    question: str = Field(description="インタビューでの質問")
    answer: str = Field(description="インタビューでの回答")


# インタビュー結果のリストを表すデータモデル
class InterviewResult(BaseModel):
    interviews: list[Interview] = Field(default_factory=list, description="インタビュー結果のリスト")


# 評価の結果を表すデータモデル
class EvaluationResult(BaseModel):
    reason: str = Field(description="判断の理由")
    is_sufficient: bool = Field(description="情報が十分かどうか")


In [3]:
# ====================================================
# 要件定義書AIエージェントのステート
# ====================================================
class InterviewState(BaseModel):
    user_request: str = Field(description="ユーザーからのリクエスト")
    personas: Annotated[list[Persona], operator.add] = Field(
        default_factory=list, description="生成されたペルソナのリスト"
    )
    interviews: Annotated[list[Interview], operator.add] = Field(
        default_factory=list, description="実施されたインタビューのリスト"
    )
    requirements_doc: str = Field(default="", description="生成された要件定義")
    iteration: int = Field(default=0, description="ペルソナ生成とインタビューの反復回数")
    is_information_sufficient: bool = Field(default=False, description="情報が十分かどうか")


## 主要コンポーネントの実装


In [4]:
# ====================================================
# PersonaGenerator：ペルソナを生成する
# ====================================================
class PersonaGenerator:
    def __init__(self, llm: ChatOpenAI, k: int = 5):
        self.llm = llm.with_structured_output(Personas)
        self.k = k

    def run(self, user_request: str) -> Personas:
        # プロンプトテンプレートを定義
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたはユーザーインタビュー用の多様なペルソナを作成する専門家です。",
                ),
                (
                    "human",
                    f"以下のユーザーリクエストに関するインタビュー用に、{self.k}人のペルソナを生成してください。\n\n"
                    "ユーザーリクエスト：{user_request}\n\n"
                    "各ペルソナには名前と簡単な背景を含めてください。年齢、性別、職業、技術的専門知識において多様性を確保してください。",
                ),
            ]
        )
        # ペルソナ作成のためのチェーンを作成
        chain = prompt | self.llm
        # ペルソナを作成
        return chain.invoke({"user_request": user_request})


In [5]:
# ====================================================
# InterviewConductor: ペルソナにインタビューを実施する
# ====================================================
class InterviewConductor:
    def __init__(self, llm: ChatOpenAI):
        self.llm = llm

    def run(self, user_request: str, personas: list[Persona]) -> InterviewResult:
        # 質問を生成
        questions = self._generate_questions(user_request=user_request, personas=personas)
        # 回答を生成
        answers = self._generate_answers(personas=personas, questions=questions)
        # 質問と回答の組合せからインタビューリストを作成
        interviews = self._create_interviews(personas=personas, questions=questions, answers=answers)
        # インタビュー結果を返す
        return InterviewResult(interviews=interviews)

    def _generate_questions(self, user_request: str, personas: list[Persona]) -> list[str]:
        # 質問生成のためのプロンプトを定義
        question_prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたはユーザー要件に基づいて、適切な質問を生成する専門家です。",
                ),
                (
                    "human",
                    "以下のペルソナに関連するユーザーリクエストについて、1つの設問を生成してください。\n\n"
                    "ユーザーリクエスト：{user_request}\n"
                    "ペルソナ：{persona_name} - {persona_background}\n\n"
                    "質問は具体的で、このペルソナの視点から重要な情報を引き出すように設計してください。",
                ),
            ]
        )
        # 質問生成のためのチェーンを作成
        question_chain = question_prompt | self.llm | StrOutputParser()

        # 各ペルソナに対する質問クエリを生成
        question_queries = [
            {
                "user_request": user_request,
                "persona_name": persona.name,
                "persona_background": persona.background,
            }
            for persona in personas
        ]
        # 質問をバッチ処理で生成
        return question_chain.batch(question_queries)

    def _generate_answers(self, personas: list[Persona], questions: list[str]) -> list[str]:
        # 回答生成のためのプロンプトを定義
        answer_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", "あなたは以下のペルソナとして回答しています：{persona_name} - {persona_background}"),
                (
                    "human",
                    "以下の質問に対して、ペルソナの視点から回答してください。\n\n" "質問：{question}\n" "回答：",
                ),
            ]
        )
        # 回答生成のためのチェーンを作成
        answer_chain = answer_prompt | self.llm | StrOutputParser()

        # 各ペルソナに対する回答クエリを作成
        answer_queries = [
            {
                "persona_name": persona.name,
                "persona_background": persona.background,
                "question": question,
            }
            for persona, question in zip(personas, questions, strict=True)
        ]
        # 回答をバッチ処理で生成
        return answer_chain.batch(answer_queries)

    def _create_interviews(self, personas: list[Persona], questions: list[str], answers: list[str]) -> list[Interview]:
        # ペルソナごとに質問と回答の組合せからインタビューオブジェクトを作成
        return [
            Interview(
                persona=persona,
                question=question,
                answer=answer,
            )
            for persona, question, answer in zip(personas, questions, answers, strict=True)
        ]


In [6]:
# ====================================================
# InformationEvaluator: 収集した情報の十分性を評価する
# ====================================================
class InformationEvaluator:
    def __init__(self, llm: ChatOpenAI):
        self.llm = llm.with_structured_output(EvaluationResult)

    def run(self, user_request: str, interviews: list[Interview]) -> EvaluationResult:
        # プロンプトを定義
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", "あなたは包括的な要件文書を作成するための情報の十分性を評価する専門家です。"),
                (
                    "human",
                    "以下のユーザーリクエストとインタビュー結果に基づいて、包括的な要件文書を作成するのに十分な情報が集まったかどうかを判断してください。\n\n"
                    "ユーザーリクエスト：{user_request}\n\n"
                    "インタビュー結果：{interview_results}",
                ),
            ]
        )
        # 情報の十分性を評価するチェーンを作成
        chain = prompt | self.llm
        # 評価結果を返す
        return chain.invoke(
            {
                "user_request": user_request,
                "interview_results": "\n".join(
                    f"ペルソナ：{interview.persona.name} - {interview.persona.background}\n"
                    f"質問：{interview.question}\n"
                    f"回答：{interview.answer}\n\n"
                    for interview in interviews
                ),
            }
        )


In [7]:
# ====================================================
# RequirementsDocumentGenerator: 要件定義書を生成する
# ====================================================
class RequirementsDocumentGenerator:
    def __init__(self, llm: ChatOpenAI):
        self.llm = llm

    def run(self, user_request: str, interviews: list[Interview]) -> str:
        # プロンプトを定義
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたは収集した情報に基づいて要件文書を作成する専門家です。",
                ),
                (
                    "human",
                    "以下のユーザーリクエストと複数のペルソナからのインタビュー結果に基づいて、要件文書を作成してください。\n\n"
                    "ユーザーリクエスト：{user_request}\n\n"
                    "インタビュー結果：{interview_results}\n"
                    "要件文書には以下のセクションを含めてください：\n"
                    "1. プロジェクト概要\n"
                    "2. 主要機能\n"
                    "3. 非機能要件\n"
                    "4. 制約条件\n"
                    "5. ターゲットユーザー\n"
                    "6. 優先順位\n"
                    "7. リスクと軽減策\n"
                    "出力は必ず日本語でお願いします。\n\n要件文書：",
                ),
            ]
        )
        # 要件文書生成のためのチェーンを作成
        chain = prompt | self.llm | StrOutputParser()
        # 要件文書を生成
        return chain.invoke(
            {
                "user_request": user_request,
                "interview_results": "\n".join(
                    f"ペルソナ：{interview.persona.name} - {interview.persona.background}\n"
                    f"質問：{interview.question}\n"
                    f"回答：{interview.answer}\n"
                    for interview in interviews
                ),
            }
        )


## ワークフロー構築

In [10]:
class DocumentationAgent:
    def __init__(self, llm: ChatOpenAI, k: int | None = None):
        # 各種ジェネレータの初期化
        self.persona_generator = PersonaGenerator(llm=llm, k=k)
        self.interview_conductor = InterviewConductor(llm=llm)
        self.information_evaluator = InformationEvaluator(llm=llm)
        self.requirements_generator = RequirementsDocumentGenerator(llm=llm)

        # グラフの作成
        self.graph = self._create_graph()

    def _create_graph(self) -> StateGraph:
        # グラフの初期化
        workflow = StateGraph(InterviewState)

        # 各ノードの作成
        workflow.add_node("generate_personas", self._generate_personas)
        workflow.add_node("conduct_interviews", self._conduct_interviews)
        workflow.add_node("evaluate_information", self._evaluate_information)
        workflow.add_node("generate_requirements", self._generate_requirements)

        # エントリーポイントの設定
        workflow.set_entry_point("generate_personas")

        # ノード間のエッジの追加
        workflow.add_edge("generate_personas", "conduct_interviews")
        workflow.add_edge("conduct_interviews", "evaluate_information")

        # 条件付きエッジの追加
        workflow.add_conditional_edges(
            "evaluate_information",
            lambda state: not state.is_information_sufficient and state.iteration < 5,
            {True: "generate_personas", False: "generate_requirements"},
        )
        workflow.add_edge("generate_requirements", END)

        # グラフのコンパイル
        return workflow.compile()

    def _generate_personas(self, state: InterviewState) -> dict[str, Any]:
        # ペルソナの生成
        new_personas: Personas = self.persona_generator.run(state.user_request)
        return {
            "personas": new_personas.personas,
            "iteration": state.iteration + 1,
        }

    def _conduct_interviews(self, state: InterviewState) -> dict[str, Any]:
        # インタビューの実施
        new_interviews: InterviewResult = self.interview_conductor.run(
            user_request=state.user_request,
            personas=state.personas[-5:],
        )
        return {
            "interviews": new_interviews.interviews,
        }

    def _evaluate_information(self, state: InterviewState) -> dict[str, Any]:
        # 情報の十分性を評価
        evaluation_result: EvaluationResult = self.information_evaluator.run(
            user_request=state.user_request,
            interviews=state.interviews,
        )
        return {
            "is_information_sufficient": evaluation_result.is_sufficient,
            "reason": evaluation_result.reason,
        }

    def _generate_requirements(self, state: InterviewState) -> dict[str, Any]:
        # 要件定義書の生成
        new_requirements: str = self.requirements_generator.run(
            user_request=state.user_request,
            interviews=state.interviews,
        )
        return {
            "requirements_doc": new_requirements,
        }

    def run(self, user_request: str) -> str:
        # 初期状態の設定
        initial_state = InterviewState(user_request=user_request)
        # グラフの実行
        final_state = self.graph.invoke(initial_state)
        # 最終的な要件定義書の取得
        return final_state["requirements_doc"]


## AIエージェントの実行

In [11]:
# ChatOpenAIモデルを初期化
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
# ドキュメント生成エージェントを初期化
agent = DocumentationAgent(llm=llm, k=None)

# ユーザーリクエストを設定
user_request = "ユーザーがログインしてメッセージを送信できるアプリを作成したい"

# エージェントの実行
final_output = agent.run(user_request)
print(final_output)


# 要件文書

## 1. プロジェクト概要
本プロジェクトは、ユーザーがログインしてメッセージを送信できるアプリを開発することを目的としています。アプリは、シンプルで直感的なユーザーインターフェースを提供し、ユーザーが快適にコミュニケーションを行える環境を整えます。特に、ビジネスシーンや日常生活での利用を考慮し、様々な機能を実装します。

## 2. 主要機能
### 2.1 ログイン機能
- **シンプルなデザイン**: ユーザー名、パスワード、ログインボタンのみを表示。
- **フィードバックの即時性**: 入力ミスに対するリアルタイムエラーメッセージ。
- **ソーシャルログインのオプション**: GoogleやFacebookアカウントを使用したログイン。
- **パスワードの可視化**: パスワード入力時に表示/非表示を切り替える機能。

### 2.2 メッセージ送信機能
- **直感的なインターフェース**: シンプルで使いやすいメッセージ送信ボックス。
- **絵文字や添付ファイルの追加**: 絵文字や画像、ファイルを簡単に添付できるボタン。
- **メッセージのプレビュー機能**: 送信前にメッセージの確認ができる機能。
- **スレッド表示**: メッセージのやり取りをスレッド形式で表示。

### 2.3 その他の機能
- **グループチャット**: 簡単にグループを作成し、メンバーを管理できる機能。
- **メッセージの消失タイマー**: 一時的な情報を共有するための機能。
- **メッセージの検索機能**: 過去のメッセージをキーワードで検索できる機能。
- **リアルタイム通知**: メッセージ受信時のプッシュ通知機能。
- **ファイル共有**: 提案書や資料を簡単に共有できる機能。

## 3. 非機能要件
- **パフォーマンス**: アプリは迅速に応答し、ログインやメッセージ送信がスムーズであること。
- **セキュリティ**: ユーザー情報の暗号化、セキュアなログインプロセスを実装。
- **ユーザビリティ**: 直感的で使いやすいインターフェースを提供し、ユーザーの学習コストを低減。
- **アクセシビリティ**: 様々なデバイス（スマートフォン、タブレット）での利用を考慮。

## 4. 制約条件
- **開発期間**: 